# 下載資料 與 DF

In [ ]:
# ame
!gdown "1KW3Y3C3wx1Eb5mZ5y6iBqKDpvUmOpbaK" --output ame.zip
!unzip ame.zip -d ame_data

# calli
!gdown "19W9Dq6lMyBE0jCdNTyHrCnD4OMUjofGu" --output calli.zip
!unzip calli.zip -d calli_data

# gura
!gdown "1ftUaR3PCkrf_fZi6ocVN4iE2NcGcmb_d" --output gura.zip
!unzip gura.zip -d gura_data

# ina
!gdown "1P1Ly9uKrbHfHivGO16HGGBVdqZ1ho-RK" --output ina.zip
!unzip ina.zip -d ina_data

# kiara
!gdown "1RoEk8VkXmqLWb1tFfAU5sIKeq4fZ6V0Q" --output kiara.zip
!unzip kiara.zip -d kiara_data

# kronii
!gdown "1UVcoPihtj241zDiPVRvj1A1U9Ew4VfFY" --output kronii.zip
!unzip kronii.zip -d kronii_data

In [ ]:
import glob
import pandas as pd


df = pd.DataFrame(columns=[ "path", "target" ])

for path in glob.glob("/content/ame_data/Watson Amelia_*.jpg"):
    s = pd.Series( [path, 0], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

for path in glob.glob("/content/calli_data/Mori Calliope_*.jpg"):
    s = pd.Series( [path, 1], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

for path in glob.glob("/content/gura_data/Gawr Gura_*.jpg"):
    s = pd.Series( [path, 2], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

for path in glob.glob("/content/ina_data/Ninomae Ina_*.jpg"):
    s = pd.Series( [path, 3], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

for path in glob.glob("/content/kiara_data/Takanashi Kiara_*.jpg"):
    s = pd.Series( [path, 4], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

for path in glob.glob("/content/kronii_data/Ouro Kronii_*.jpg"):
    s = pd.Series( [path, 5], index=[ "path", "target" ])
    df = df.append(s, ignore_index=True)

In [7]:
df

,path,target
0,/content/ame_data/Watson Amelia_52.jpg,0
1,/content/ame_data/Watson Amelia_63.jpg,0
2,/content/ame_data/Watson Amelia_66.jpg,0
3,/content/ame_data/Watson Amelia_59.jpg,0
4,/content/ame_data/Watson Amelia_67.jpg,0
...,...,...
1444,/content/kronii_data/Ouro Kronii_76.jpg,5
1445,/content/kronii_data/Ouro Kronii_190.jpg,5
1446,/content/kronii_data/Ouro Kronii_14.jpg,5
1447,/content/kronii_data/Ouro Kronii_204.jpg,5


# 隨機檢查圖片

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import random


random_index = random.randint(0, 1448)

print(df["path"][random_index])
load_img(df["path"][random_index])

# 模型

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout


model_VGG = VGG16( include_top=False, input_shape=(250, 250, 3) )

# 參考VGG16 不訓練某些權重
for layer in model_VGG.layers:
    layer.trainable=False

# 
model = Sequential([
    model_VGG,
    BatchNormalization(),
    GlobalAveragePooling2D(),  # 512
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(6, activation="softmax")
])

58889256/58889256 [==============================] - 4s 0us/step


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8

In [12]:
model.compile( loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np


# to_categorical
y_categorical = to_categorical(df["target"])

x_train, x_test, y_train, y_test = train_test_split( np.array(df["path"]),
                               np.array(y_categorical),
                               test_size=0.2)

In [14]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1159,), (290,), (1159, 6), (290, 6))

In [15]:
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rotation_range=25,
                 width_shift_range=0.35,
                 height_shift_range=0.35,
                 zoom_range=0.25,
                 horizontal_flip=True)

batch_size = 200

for i in range(50):
    idx = np.random.randint( 0, x_train.shape[0], batch_size )

    imglist = []

    for fn in x_train[idx]:
        img = load_img(fn, target_size=(250, 250, 3))
        img = img.resize((250, 250))            # 將圖像大小調整為 250x250
        img = np.array(img)                 # 轉換為 NumPy array
        img = datagen.flow(np.expand_dims(img, axis=0))  # 輸入前必須擴張一個 batch 的維度 (之前只讀取一張圖片 250*250*3 但這邊需要批次處理 1*250*250*3)
        img = next(img)[0]                  # 從生成器中獲取一張圖像，並且刪除 batch 的維度
        img = preprocess_input(img)             # 預處理圖像
        imglist.append(img)

    x = np.array(imglist)
    y = y_train[idx]
    result = model.train_on_batch(x, y) # train_on_batch 會輸出: loss值, 正確率
    print("-----Times", i, "-----")
    print(result)

-----Times 0 -----
[1.8397349119186401, 0.1733333319425583]
-----Times 1 -----
[1.787770390510559, 0.20666666328907013]
-----Times 2 -----
[1.7414495944976807, 0.2266666740179062]
-----Times 3 -----
[1.6842113733291626, 0.30666667222976685]
-----Times 4 -----
[1.6378909349441528, 0.3266666531562805]
-----Times 5 -----
[1.5981718301773071, 0.40666666626930237]
-----Times 6 -----
[1.5531703233718872, 0.36666667461395264]
-----Times 7 -----
[1.506839394569397, 0.4333333373069763]
-----Times 8 -----
[1.4471266269683838, 0.5400000214576721]
-----Times 9 -----
[1.3974055051803589, 0.6000000238418579]
-----Times 10 -----
[1.3122014999389648, 0.6399999856948853]
-----Times 11 -----
[1.3464401960372925, 0.6066666841506958]
-----Times 12 -----
[1.3204725980758667, 0.6266666650772095]
-----Times 13 -----
[1.2190111875534058, 0.7266666889190674]
-----Times 14 -----
[1.2062596082687378, 0.7666666507720947]
-----Times 15 -----
[1.1270540952682495, 0.7200000286102295]
-----Times 16 -----
[1.027390122

In [16]:
batch_size = 200

idx = np.random.randint( 0, x_test.shape[0], batch_size )

imglist = []

for fn in x_test[idx]:
    img = load_img( fn, target_size=(250, 250, 3) )
    img = preprocess_input( np.array(img) )
    imglist.append(img)

x = np.array(imglist)
y = y_test[idx]
model.evaluate(x, y)

10/10 [==============================] - 9s 406ms/step - loss: 0.2803 - accuracy: 0.9100


[0.280305415391922, 0.9100000262260437]

In [18]:
predicted_vtuber = model.predict(x).argmax(axis=-1)
# print(predicted_vtuber)
actual_vtuber = np.array(y).argmax(axis=-1)
# print(actual_vtuber)

10/10 [==============================] - 2s 162ms/step
[5 5 2 5 4 1 1 4 3 0 4 4 0 2 4 4 0 5 1 3 1 1 5 1 4 1 4 2 5 1 4 0 2 4 5 5 4
 4 1 4 4 5 0 0 4 1 0 4 1 4 5 4 0 2 3 2 0 1 0 4 0 4 5 3 4 4 0 1 4 5 4 5 2 2
 0 3 1 3 4 4 3 4 4 5 5 0 1 3 3 1 1 2 4 4 1 3 0 2 5 0 5 4 1 4 3 2 4 1 4 4 5
 3 2 0 3 2 2 4 3 1 0 4 3 1 4 4 0 0 5 2 2 3 3 3 3 3 1 2 4 2 4 1 2 4 2 3 4 5
 3 2 0 3 5 2 5 0 0 4 2 5 5 3 0 4 2 4 2 0 2 5 0 5 3 3 0 2 2 0 2 1 1 1 4 5 2
 3 4 4 3 0 4 3 0 3 2 3 1 4 0 4 0 5 3 4 4 5 5 5 3 1 4 1 1 3 4 4 1 0 3 1 4 3
 5 4 2 2 3 2 3 4 5 5 2 4 5 3 3 3 4 4 3 0 4 4 2 3 4 4 2 5 0 5 2 4 4 4 1 0 4
 1 0 1 4 5 1 1 4 5 2 1 5 2 2 4 3 1 3 3 1 4 2 2 0 4 5 5 5 4 0 2 4 0 3 3 4 3
 0 0 4 3]
[5 5 2 5 3 1 1 4 3 0 0 4 0 2 4 4 0 5 1 3 1 1 5 1 4 1 4 2 5 1 4 0 2 0 5 5 4
 4 1 3 4 5 0 0 4 1 0 4 1 3 5 4 0 2 3 2 0 1 0 0 0 4 5 3 4 4 0 1 3 5 4 5 2 2
 2 3 1 3 1 4 3 5 4 5 5 0 1 3 3 1 1 2 4 4 1 3 0 2 5 0 5 1 1 4 3 2 4 1 4 4 5
 3 2 0 3 2 2 4 5 1 0 4 3 1 4 3 0 0 5 2 2 3 3 3 3 3 1 2 4 2 0 1 2 4 2 3 1 5
 3 2 0 3 5 2 5 0 0 4 2 5 5 3 0 5 2 

In [19]:
from sklearn.metrics import confusion_matrix


trans= ["Ame", "Calli", "Gura", "Ina", "Kiara", "Kronii"]


mat = confusion_matrix(actual_vtuber, predicted_vtuber)
pd.DataFrame(mat,
       index=["{}(true)".format(trans[i]) for i in range(6)],
       columns=["{}(pred)".format(trans[i]) for i in range(6)])

,Ame(pred),Calli(pred),Gura(pred),Ina(pred),Kiara(pred),Kronii(pred)
Ame(true),40,0,0,0,9,0
Calli(true),0,41,0,0,5,0
Gura(true),2,0,44,0,0,0
Ina(true),0,0,0,46,5,0
Kiara(true),0,0,0,0,59,0
Kronii(true),0,0,0,4,2,43


# 儲存模型

In [20]:
from tensorflow.keras.models import load_model


model.save('Vtuber_Identify_(tf_vgg16).h5')

In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np


model = load_model("Vtuber_Identify_(tf_vgg16).h5")

img_path = "/content/ina_8.jpg"
img = image.load_img(img_path, target_size=(250, 250, 3))

x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)

preds = model.predict(x)

vtuber_list = ["ame", "calli", "gura", "ina", "kiara", "kronii"]


print(f"預測Vtuber為: {vtuber_list[preds.argmax()]}")

1/1 [==============================] - 0s 160ms/step
預測Vtuber為: ina
